In [2]:
import ivolatility as ivol
import pandas as pd
from auth import initializeIvolAuthentication

initializeIvolAuthentication()

# Let's now imagine that we are interested in extracting options on futures from the previous example No.5
getFutData = ivol.setMethod('/futures/eod/futures-info')
futDataRaw = getFutData(date='2023-11-01',underlyingName='E%Mini%S%P%500%')

futuresNeed = getFutData(date='2023-11-01',rootSymbol='ES', MIC = 'XCME')

# Let's now find the prices for these futures for period from '2023-11-01' to '2023-11-10'
geFutPriceData = ivol.setMethod('/futures/single-futures')

# finalFutData = pd.DataFrame()
# for futSymbol in futuresNeed['Futsymbol']:
#   futPriceData = geFutPriceData(symbol=futSymbol, from_='2023-11-01', to='2023-11-10')
#   finalFutData = pd.concat([finalFutData, futPriceData], axis=0) 

finalFutData = pd.DataFrame()
for futSymbol in futuresNeed['FutSymbol']:  # Corrected column name
    futPriceData = geFutPriceData(symbol=futSymbol, from_='2023-11-01', to='2023-11-10')
    finalFutData = pd.concat([finalFutData, futPriceData], axis=0)
    
'''
Creating a list of required futures (take from the previous example) 
with expiration dates between "2023-10-21" and "2024-10-21"
'''

myFutList = finalFutData['Future symbol'][(finalFutData['Future expiration'] > '2023-10-21') & (finalFutData['Future expiration'] < '2024-10-21')].unique()

futOptsList = pd.DataFrame()

'''
Suppose we need call options on ES futures also with expiration dates between "2023-10-21" and "2024-10-21", 
a strike range between 3800 and 3900 and on the date "2023-11-01"
'''

# The process of extracting option tickers based on chosen parameters
getFutOptsData = ivol.setMethod('/futures/eod/option-series-on-date')
for futSymbol in myFutList:
  futOptsNeed = getFutOptsData(date='2023-11-01', futuresSymbol=futSymbol, expFrom='2023-10-21', expTo='2024-10-21', strikeFrom=3800, strikeTo=3900, callPut='C')
  futOptsList = pd.concat([futOptsList, futOptsNeed], axis=0)

finalOptsCalcData = pd.DataFrame()

# Getting futures options pricing data with implied volatilities and Greeks
getFutOptsCalcData = ivol.setMethod('/futures/eod/single-future-option-rawiv')
for futOptSymbol in futOptsList['opt_symbol']:
  futOptsCalcData = getFutOptsCalcData(symbol=futOptSymbol, from_='2023-11-01', to='2023-11-10')
  finalOptsCalcData = pd.concat([finalOptsCalcData, futOptsCalcData], axis=0)

finalOptsCalcData

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))